In [ ]:
import ftplib
import os
import tkinter as tk
from tkinter import filedialog,font

class FTPClient:
    def __init__(self, host, port, user, password):
        self.host = host
        self.port = port
        self.user = user
        self.password = password
        self.ftp = ftplib.FTP()
        
    def connect(self):
        try:
            self.ftp.connect(self.host, self.port)
            self.ftp.login(self.user, self.password)
        except Exception as e:
            print(f"Failed to connect to FTP server: {e}")
            traceback.print_exc()

        
    def disconnect(self):
        self.ftp.quit()
        
    def upload(self, local_file, remote_file):
        with open(local_file, 'rb') as f:
            self.ftp.storbinary(f'STOR {remote_file}', f)

    def download(self, remote_file, local_file):
        with open(local_file, "wb") as f:
            self.ftp.retrbinary(f"RETR {remote_file}", f.write)
            
    def list_files(self):
        return self.ftp.nlst()
    
    
    
class FTPClientGUI:
    def __init__(self, master, client):
        self.client = client
        def read_users_from_file(file_name):
            users = []
            with open(file_name) as f:
                for line in f:
                    user_info = line.strip().split(",")
                    users.append({"username": user_info[0], "password": user_info[1], "permissions": user_info[2]})
            return users
        
        self.is_student = False
        self.is_teacher = False
        # Read users from the students file and create them with limited permissions
        self.students = read_users_from_file("students.txt")
        self.teachers = read_users_from_file("teachers.txt")

            
    
        # Colors
        bg_color = "#1b1917"
        label_color = "#e4e4e4"
        button_color = "#6e499a"
        button_text_color = "#fff"

        # Fonts
        label_font = font.Font(family="Helvetica", size=12, weight="bold")
        entry_font = font.Font(family="Helvetica", size=12)
        button_font = font.Font(family="Helvetica", size=12, weight="bold")

        # Host input
        self.host_label = tk.Label(master, text="Host:", bg=bg_color, fg=label_color, font=label_font,)
        self.host_label.grid(row=0, column=0, sticky=tk.W, padx=20, pady=20)
        self.host_entry = tk.Entry(master, font=entry_font)
        self.host_entry.grid(row=0, column=1, padx=20, pady=20)

        # Port input
        self.port_label = tk.Label(master, text="Port:", bg=bg_color, fg=label_color, font=label_font)
        self.port_label.grid(row=1, column=0, sticky=tk.W, padx=20, pady=20)
        self.port_entry = tk.Entry(master, font=entry_font)
        self.port_entry.grid(row=1, column=1, padx=20, pady=20)

        # Username input
        self.user_label = tk.Label(master, text="Username:", bg=bg_color, fg=label_color, font=label_font)
        self.user_label.grid(row=2, column=0, sticky=tk.W, padx=20, pady=20)
        self.user_entry = tk.Entry(master, font=entry_font)
        self.user_entry.grid(row=2, column=1, padx=20, pady=20)

        # Password input
        self.password_label = tk.Label(master, text="Password:", bg=bg_color, fg=label_color, font=label_font)
        self.password_label.grid(row=3, column=0, sticky=tk.W, padx=20, pady=20)
        self.password_entry = tk.Entry(master, show="*", font=entry_font)
        self.password_entry.grid(row=3, column=1, padx=20, pady=20)

        # Connect button
        self.connect_button = tk.Button(master, text="Connect", bg=button_color, fg=button_text_color, font=button_font, command=self.connect)
        self.connect_button.grid(row=4, column=1, padx=20, pady=20)

        # Connection status label
        self.connected_label = tk.Label(master, text="FTP server status: Not connected", bg=bg_color, fg="#d70f33", font=label_font)
        self.connected_label.grid(row=5, column=1, padx=20, pady=20)

        # Create a new window for the FTP actions
        self.ftp_window = tk.Toplevel(master)
        self.ftp_window.withdraw()
        self.ftp_window.configure(bg="#1b1917")

        # Fonts
        label_font_small = font.Font(family="Helvetica", size=12, weight="bold")
        button_font_small = font.Font(family="Helvetica", size=12, weight="bold")
        
        self.local_file_label = tk.Label(self.ftp_window, text="Local file:",bg=bg_color, fg=label_color, font=label_font_small )
        self.local_file_label.grid(row=0, column=0, sticky=tk.W, padx=20, pady=20)

        self.local_file_entry = tk.Entry(self.ftp_window,  font=entry_font)
        self.local_file_entry.grid(row=0, column=1, padx=20, pady=20)

        self.local_file_button = tk.Button(self.ftp_window, text="Browse", command=self.browse_local_file, bg=button_color, fg=button_text_color, font=button_font_small, width=10)
        self.local_file_button.grid(row=0, column=2, padx=20, pady=20)

        self.remote_file_label = tk.Label(self.ftp_window, text="Remote file:", bg=bg_color, fg=label_color, font=label_font_small)
        self.remote_file_label.grid(row=1, column=0, sticky=tk.W, padx=20, pady=20)

        self.remote_file_entry = tk.Entry(self.ftp_window, font=entry_font)
        self.remote_file_entry.grid(row=1, column=1, padx=20, pady=20)

        self.upload_button = tk.Button(self.ftp_window, text="Upload file", bg=button_color, fg=button_text_color, font=button_font_small, command=self.upload, state=tk.DISABLED, width=15)
        self.upload_button.grid(row=2, column=0, padx=20, pady=20)

        self.file_list_label = tk.Label(self.ftp_window, text="Remote files:", bg=bg_color, fg=label_color, font=label_font_small)
        self.file_list_label.grid(row=3, column=0, sticky=tk.W, padx=20, pady=20)

        # Create listbox with scrollbar
        self.file_listbox = tk.Listbox(self.ftp_window, font=label_font_small)
        self.file_scrollbar = tk.Scrollbar(self.ftp_window, command=self.file_listbox.yview)
        self.file_listbox.config(yscrollcommand=self.file_scrollbar.set)
        self.file_listbox.grid(row=4, column=0, columnspan=2, padx=20, pady=10)
        self.file_scrollbar.grid(row=4, column=1, sticky='ns', pady=10)

        # Create toggle button for file details
        self.show_details = tk.BooleanVar()
        self.show_details.set(False)
        self.detail_button = tk.Checkbutton(self.ftp_window, text="Show Details", variable=self.show_details)
        self.detail_button.grid(row=5, column=0, padx=20, pady=10)

        self.refresh_file_list_button = tk.Button(self.ftp_window, text="Refresh", bg=button_color, fg=button_text_color, font=button_font_small, command=self.refresh_file_list, state=tk.DISABLED, width=15)
        self.refresh_file_list_button.grid(row=5, column=0, padx=20, pady=20)

        self.disconnect_button = tk.Button(self.ftp_window, text="Disconnect", bg=button_color, fg=button_text_color, font=button_font_small, command=self.disconnect, state=tk.DISABLED, width=15)
        self.disconnect_button.grid(row=5, column=1, padx=20, pady=20)

    def connect(self):
        host = self.host_entry.get()
        port = int(self.port_entry.get())
        user = self.user_entry.get()
        password = self.password_entry.get()
 
        self.client.host = host
        self.client.port = port
        self.client.user = user
        self.client.password = password
        for student in self.students:
            if client.user in student.values() and client.password in student.values():
                self.is_student = True
        for teacher in self.teachers:
            if client.user in teacher.values() and client.password in teacher.values():
                self.is_teacher = True 
            print(client.user,teacher.values(),client.password )
        print(self.is_teacher)
        
        if self.is_teacher:
            self.download_button = tk.Button(self.ftp_window, text="Download file", bg="#6e499a", fg="#fff", font=font.Font(family="Helvetica", size=12, weight="bold"), command=self.download, state=tk.DISABLED, width=15)
            self.download_button.grid(row=2, column=1, padx=20, pady=20)
            self.download_button.config(state=tk.NORMAL)
            self.essay_button = tk.Button(self.ftp_window, text="Grade essays", bg="#6e499a", fg="#fff", font=font.Font(family="Helvetica", size=12, weight="bold"), command=self.download, state=tk.DISABLED, width=15)
            self.essay_button.grid(row=2, column=2, padx=20, pady=20)
            self.essay_button.config(state=tk.NORMAL)

        try:
            self.client.connect()
        except:
            tk.messagebox.showerror("Connection Error", "Could not connect to FTP server")
            return

        self.connected_label.config(text="FTP server status: Connected",fg="#27b10e")
        self.connect_button.config(state=tk.DISABLED)
        self.disconnect_button.config(state=tk.NORMAL)
        self.upload_button.config(state=tk.NORMAL)
        self.refresh_file_list_button.config(state=tk.NORMAL)

        self.refresh_file_list()

        # Show the FTP actions window
        self.ftp_window.deiconify()

    def disconnect(self):
        self.client.disconnect()
        self.connected_label.config(text="FTP server status: Not connected")
        self.connect_button.config(state=tk.NORMAL)
        self.disconnect_button.config(state=tk.DISABLED)
        self.upload_button.config(state=tk.DISABLED)
        self.download_button.config(state=tk.DISABLED)
        self.refresh_file_list_button.config(state=tk.DISABLED)
        self.file_listbox.delete(0, tk.END)

        # Hide the FTP actions window
        self.ftp_window.withdraw()

    def upload(self):
        local_file = self.local_file_entry.get()
        remote_file = self.remote_file_entry.get()
        self.client.upload(local_file, remote_file)

        tk.messagebox.showinfo("Upload Successful", "File uploaded successfully")
        self.local_file_entry.delete(0, tk.END)
        self.refresh_file_list()

    def download(self):
        remote_file = self.file_listbox.get(tk.ACTIVE)
        local_file = filedialog.asksaveasfilename(initialfile=remote_file)

        try:
            self.client.download(remote_file, local_file)
        except:
            tk.messagebox.showerror("Download Error", "Could not download file")
            return

        tk.messagebox.showinfo("Download Successful", "File downloaded successfully")

    def refresh_file_list(self):
        self.file_listbox.delete(0, tk.END)
        files = self.client.list_files()
        for file in files:
            self.file_listbox.insert(tk.END, file)
            
    def browse_local_file(self):
filetypes = (("Image files", "*.jpg;*.jpeg;*.png;*.gif"),)
        filename = filedialog.askopenfilename(filetypes=filetypes)
        if filename:
            self.local_file_entry.delete(0, tk.END)
            self.local_file_entry.insert(0, filename)
            self.remote_file_entry.insert(0, os.path.basename(filename))
        
root = tk.Tk()
root.geometry("500x500")
root.title("FTP Client")
root.configure(bg="#1b1917")

client = FTPClient("127.0.0.1", 21, "bahadur", password="123")
app = FTPClientGUI(root, client)



root.mainloop()


john123 dict_values(['professor1', 'password4', 'elradfmwM']) password1
john123 dict_values(['instructor2', 'password5', 'elradfmwM']) password1
john123 dict_values(['lecturer3', 'password6', 'elradfmwM']) password1
False
Failed to connect to FTP server: [WinError 10061] No connection could be made because the target machine actively refused it
john123 dict_values(['professor1', 'password4', 'elradfmwM']) password1
john123 dict_values(['instructor2', 'password5', 'elradfmwM']) password1
john123 dict_values(['lecturer3', 'password6', 'elradfmwM']) password1
False
